In [1]:
import os
import mido
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
import numpy as np
from utils.preprocess.quantize_note_timings import quantize_note_timings
from utils.preprocess.normalize_velocities import normalize_velocities
from utils.preprocess.filter_unnecessary_data import filter_unnecessary_data
from utils.train.functions import split_train_validation_data, preprocess_data_for_training
from utils.predict.functions import preprocess_data_for_prediction, postprocess_sequences_to_midi
from utils.impure.functions import process_directory
from music21 import *


2023-05-28 15:50:13.303394: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-28 15:50:13.304834: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 15:50:13.335036: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 15:50:13.335628: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 15:50:13.965260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Read and parse MIDI file using mido
def parse_midi_file(midi_file_path):
    midi_data = mido.MidiFile(midi_file_path)
    return midi_data


In [3]:
# Preprocess MIDI data before feeding it to the machine learning model
def preprocess_midi_data(midi_data):
    midi_data = quantize_note_timings(midi_data)
    midi_data = normalize_velocities(midi_data)
    midi_data = filter_unnecessary_data(midi_data)
    return midi_data

In [4]:
# Load a list of MIDI files for training and validation
def load_midi_files(file_directory):
    midi_files = []
    for root, dirs, files in os.walk(file_directory):
        for file in files:
            if file.endswith(".mid") or file.endswith(".midi"):
                midi_files.append(os.path.join(root, file))

    return midi_files

In [5]:

def train_lstm_model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes):

    print("Train sequences shape: ", train_sequences.shape)
    print("Train labels shape: ", train_labels.shape)
    print("Validation sequences shape: ", validation_sequences.shape)
    print("Validation labels shape: ", validation_labels.shape)
    print("Input shape: ", input_shape)
    print("Number of classes: ", num_classes)

    # Define the LSTM model architecture
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.LSTM(units=128, return_sequences=True),
        tf.keras.layers.LSTM(units=128),
        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    ])

    print(model.summary())
    print("Model input shape: ", model.input_shape)
    print("Model output shape: ", model.output_shape)
    

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Fit the model to the data
    history = model.fit(train_sequences, train_labels, epochs=20, validation_data=(validation_sequences, validation_labels))

    return model, history

In [42]:
def train_machine_learning_model():
    # Load and preprocess my MIDI data
    pure_file_directory = "./adl-piano-midi/Children"
    impure_file_directory = "./adl-piano-midi-impure/Children"
    
    pure_midi_files = load_midi_files(pure_file_directory)
    impure_midi_files = load_midi_files(impure_file_directory)
    
    print("Loaded {} pure MIDI files".format(len(pure_midi_files)))
    print("Loaded {} impure MIDI files".format(len(impure_midi_files)))

    preprocessed_pure_midi_data = [preprocess_midi_data(parse_midi_file(file)) for file in pure_midi_files]
    preprocessed_impure_midi_data = [preprocess_midi_data(parse_midi_file(file)) for file in impure_midi_files]
    
    print("Preprocessed {} pure MIDI files".format(len(preprocessed_pure_midi_data)))
    print("Preprocessed {} impure MIDI files".format(len(preprocessed_impure_midi_data)))

    # Split the preprocessed MIDI data into training and validation sets
    train_data, validation_data, train_labels, validation_labels = split_train_validation_data(preprocessed_impure_midi_data, preprocessed_pure_midi_data)
    
    print("Split {} MIDI files into {} training files and {} validation files".format(len(preprocessed_impure_midi_data), len(train_data), len(validation_data)))

    # Further preprocess the MIDI data to create input sequences and corresponding labels for training
    train_sequences, train_labels = preprocess_data_for_training(train_data, train_labels)
    validation_sequences, validation_labels = preprocess_data_for_training(validation_data, validation_labels)
    
    print("Created {} training sequences and {} validation sequences".format(len(train_sequences), len(validation_sequences)))

    # Determine input_shape and num_classes based on preprocessed data
    input_shape = train_sequences.shape[1:] # Use the shape of the sequences for input_shape
    num_classes = np.max(train_labels) + 1 # Determine the number of unique classes in train_labels

    # Train the LSTM model
    model, history = train_lstm_model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes)
    return model, history

In [40]:
def postprocess_sequences_to_midi(sequence, output_file_path):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    for note_sequence in sequence:
        for note in note_sequence:
            # Ensure the note value is an integer
            note = int(note)

            note_on = mido.Message('note_on', note=note, velocity=64, time=0)
            track.append(note_on)

            note_off = mido.Message('note_off', note=note, velocity=64, time=960)
            track.append(note_off)

    mid.save(output_file_path)

def filter_midi_data(midi_data, model, sequence_length=32, midi_file_path="./resultMIDI/impure.mid"):    # Preprocess the MIDI data to get input sequences
    input_sequences = preprocess_data_for_prediction(midi_data, sequence_length)
    print("Created {} input sequences".format(len(input_sequences)))
    print("Input sequences shape: ", input_sequences.shape)

    # Use the model to predict the clean sequences
    clean_sequences = model(input_sequences)
    print("Clean sequences shape: ", clean_sequences.shape)
    print("Clean sequences: ", clean_sequences)
    # Postprocess the clean sequences to get a clean MIDI file
    clean_midi_data = postprocess_sequences_to_midi(clean_sequences, "./resultMIDI/" + os.path.splitext(os.path.basename(midi_file_path))[0] + ".mid")

    return clean_midi_data

In [8]:
def convert_midi_to_music_representation(midi_data):
    # Convert MIDI data to music representation using music21
    music_rep = converter.parse(midi_data)
    return music_rep

In [9]:
def generate_sheet_music(music_representation):
    sheet_music = music_representation
    return sheet_music

In [10]:
def export_sheet_music(sheet_music, output_format, filename):
    # Export sheet music to desired format (e.g., PDF)
    sheet_music.write(output_format, fp=filename)

In [11]:
if __name__ == "__main__":
    model = train_machine_learning_model()
    print("Model trained, summary: ", model[0].summary())
    #Make an impure version of the midi file directory and save it to adl-piano-midi-impure
    #process_directory("./adl-piano-midi")

Loaded 24 pure MIDI files
Loaded 24 impure MIDI files
Preprocessed 24 pure MIDI files
Preprocessed 24 impure MIDI files
Split 24 MIDI files into 19 training files and 5 validation files
Created 38267 training sequences and 8619 validation sequences
Train sequences shape:  (38267, 32, 2)
Train labels shape:  (38267,)
Validation sequences shape:  (8619, 32, 2)
Validation labels shape:  (8619,)
Input shape:  (32, 2)
Number of classes:  128


2023-05-28 15:50:19.149273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 15:50:19.149585: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32, 128)           67072     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
Total params: 215,168
Trainable params: 215,168
Non-trainable params: 0
_________________________________________________________________
None
Model input shape:  (None, 32, 2)
Model output shape:  (None, 128)
Epoch 1/20


2023-05-28 15:50:19.440184: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 15:50:19.441043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-28 15:50:19.441757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1195/1196 [============================>.] - ETA: 0s - loss: 3.6265 - accuracy: 0.0983

2023-05-28 15:50:47.267203: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 15:50:47.268731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-28 15:50:47.269727: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1196/1196 [==============================] - 30s 23ms/step - loss: 3.6266 - accuracy: 0.0983 - val_loss: 3.7351 - val_accuracy: 0.0750
Epoch 2/20
1196/1196 [==============================] - 26s 22ms/step - loss: 3.3772 - accuracy: 0.1231 - val_loss: 3.5420 - val_accuracy: 0.0862
Epoch 3/20
1196/1196 [==============================] - 26s 22ms/step - loss: 3.2582 - accuracy: 0.1423 - val_loss: 3.6620 - val_accuracy: 0.0834
Epoch 4/20
1196/1196 [==============================] - 27s 22ms/step - loss: 3.1734 - accuracy: 0.1584 - val_loss: 3.6568 - val_accuracy: 0.0936
Epoch 5/20
1196/1196 [==============================] - 27s 22ms/step - loss: 3.0824 - accuracy: 0.1757 - val_loss: 3.6895 - val_accuracy: 0.0832
Epoch 6/20
1196/1196 [==============================] - 27s 22ms/step - loss: 3.0109 - accuracy: 0.1893 - val_loss: 3.7140 - val_accuracy: 0.0823
Epoch 7/20
1196/1196 [==============================] - 27s 22ms/step - loss: 2.9416 - accuracy: 0.2050 - val_loss: 3.7006 - val_accura

In [12]:
root = tk.Tk()
root.withdraw()  # we don't want a full GUI, so keep the root window from appearing
midi_file_path = filedialog.askopenfilename()  # show an "Open" dialog box and return the path to the selected file
midi_data = parse_midi_file(midi_file_path)
preprocess_midi_data(midi_data)

MidiFile(type=1, ticks_per_beat=240, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=833333, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)]),
  MidiTrack([
    Message('note_on', channel=0, note=40, velocity=42, time=1920),
    Message('note_on', channel=0, note=60, velocity=17, time=0),
    Message('note_on', channel=0, note=59, velocity=105, time=0),
    Message('note_on', channel=0, note=64, velocity=68, time=69600),
    Message('note_on', channel=0, note=52, velocity=48, time=180),
    Message('note_on', channel=0, note=59, velocity=0, time=30),
    Message('note_on', channel=0, note=56, velocity=82, time=86),
    Message('note_on', channel=0, note=59, velocity=82, time=60),
    Message('note_on', channel=0, note=56, velocity=0, time=28),
    Message('note_on', channel=0, note=52, velocity=0, time=12),
    Message('note_on', channel=0, note=40, velocity=0, time=18),
    Message('note_on',

In [41]:
print("Filtering MIDI data...")
clean_midi = filter_midi_data(midi_data, model[0], 32, midi_file_path)
print("Filtered MIDI data")

Filtering MIDI data...
Reshaped input sequences shape:  (2227, 32, 2)
Reshaped input sequences:  [[[    0     0]
  [    0     0]
  [    0     0]
  ...
  [   59     0]
  [   61     0]
  [   64     0]]

 [[    0     0]
  [    0     0]
  [    0     1]
  ...
  [   61     0]
  [   64     0]
  [   64 66720]]

 [[    0     0]
  [    0     1]
  [    0     0]
  ...
  [   64     0]
  [   64 66720]
  [    0    60]]

 ...

 [[   58     0]
  [   58   480]
  [   59 22560]
  ...
  [   61   600]
  [   59   480]
  [   59   480]]

 [[   58   480]
  [   59 22560]
  [   41   480]
  ...
  [   59   480]
  [   59   480]
  [   59     0]]

 [[   59 22560]
  [   41   480]
  [   59 22080]
  ...
  [   59   480]
  [   59     0]
  [   59   480]]]
Filtered MIDI data


In [ ]:
print("Converting MIDI to music representation...")
music_representation = convert_midi_to_music_representation(clean_midi)
sheet_music = generate_sheet_music(music_representation)
output_file_path_XML = "./resultXML/" + os.path.splitext(os.path.basename(midi_file_path))[0] + ".xml"
#output_file_path_PDF = "./resultPDF/" + os.path.splitext(os.path.basename(midi_file_path))[0] + ".pdf"
export_sheet_music(sheet_music, "musicxml", output_file_path_XML)
print("Exported sheet music to XML to ", output_file_path_XML)